In [1]:
import $file.^.Magic

import $file.$

In [2]:
val zipName = "sms+spam+collection.zip"
val datasetUrl = s"https://archive.ics.uci.edu/static/public/228/$zipName"
val outputDir = "data/sms-spam-raw"

zipName: String = "sms+spam+collection.zip"
smsSpamUrl: String = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
outputDir: String = "data/sms-spam-raw"

In [3]:
Magic.!("curl", "--create-dirs", "-O", "--output-dir", outputDir, datasetUrl)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 36789    0 36789    0     0  48504      0 --:--:-- --:--:-- --:--:-- 48470
100  198k    0  198k    0     0   176k      0 --:--:--  0:00:01 --:--:--  176k


In [4]:
Magic.!("unzip", s"$outputDir/$zipName", "-d", outputDir)

Archive:  data/sms-spam-raw/sms+spam+collection.zip
  inflating: data/sms-spam-raw/SMSSpamCollection  
  inflating: data/sms-spam-raw/readme  


In [17]:
import scala.io.Source

val datasetRaw = Source.fromFile(s"$outputDir/SMSSpamCollection").mkString

case class SmsSpamRecord(
  text: String,
  isSpam: Boolean
)

type Dataset = Vector[SmsSpamRecord]

val smsSpamRecords: Dataset = datasetRaw.split("\n").map {
  case s"spam\t$text" => SmsSpamRecord(text, isSpam = true)
  case s"ham\t$text" => SmsSpamRecord(text, isSpam = false)
}.toVector

val (spamRecords, notSpamRecords) = smsSpamRecords.partition(_.isSpam)
println(s"Spam count: ${spamRecords.size}")
println(s"Not spam count: ${notSpamRecords.size}")

Spam count: 747
Not spam count: 4827


import scala.io.Source
datasetRaw: String = """ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461

In [18]:
import scala.collection.mutable
import scala.util.Random

val balancedDataset: Dataset = {

  def sample(records: Vector[SmsSpamRecord], targetSize: Int): Vector[SmsSpamRecord] = {
    val balancedDatasetSpam = mutable.Map[String, SmsSpamRecord]()
    while (balancedDatasetSpam.size < targetSize) {
      val randomRecord = records(Random.nextInt(records.size))
      if (!balancedDatasetSpam.contains(randomRecord.text))
        balancedDatasetSpam += randomRecord.text -> randomRecord
    }
    balancedDatasetSpam.values.toVector
  }

  if (spamRecords.size < notSpamRecords.size)
    spamRecords ++ sample(notSpamRecords, targetSize = spamRecords.size)
  else
    notSpamRecords ++ sample(spamRecords, targetSize = notSpamRecords.size)
}

import scala.collection.mutable
import scala.util.Random
balancedDataset: Dataset = Vector(
  SmsSpamRecord(
    text = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "SIX chances to win CASH! From 100 to 20,000 po

In [22]:
type Training = Dataset
type Validation = Dataset
type Test = Dataset

def randomSplit(dataset: Vector[SmsSpamRecord], trainingFraction: Double, validationFraction: Double): (Training, Validation, Test) = {
  val shuffledDataset = Random.shuffle(dataset)
  val trainingSize = (shuffledDataset.size * trainingFraction).floor.toInt
  val validationSize = (shuffledDataset.size * validationFraction).floor.toInt

  val (training, remainingRecords) = shuffledDataset.splitAt(trainingSize)
  val (validation, test) = remainingRecords.splitAt(validationSize)
  (training, validation, test)
}

val (training, validation, test) = randomSplit(balancedDataset, trainingFraction = 0.7, validationFraction = 0.1) 

defined type Training
defined type Validation
defined type Test
defined function randomSplit
training: Training = Vector(
  SmsSpamRecord(
    text = "* FREE* POLYPHONIC RINGTONE Text SUPER to 87131 to get your FREE POLY TONE of the week now! 16 SN PoBox202 NR31 7ZS subscription 450pw",
    isSpam = true
  ),
  SmsSpamRecord(text = "Do you know when the result.", isSpam = false),
  SmsSpamRecord(text = "Aight, we'll head out in a few", isSpam = false),
  SmsSpamRecord(
    text = "Hi, Mobile no.  &lt;#&gt;  has added you in their contact list on www.fullonsms.com It s a great place to send free sms to people For more visit fullonsms.com",
    isSpam = false
  ),
  SmsSpamRecord(text = "You will go to walmart. I.ll stay.", isSpam = false),
  SmsSpamRecord(
    text = "Hi, this is Mandy Sullivan calling from HOTMIX FM...you are chosen to receive £5000.00 in our Easter Prize draw.....Please telephone 09041940223 to claim before 29/03/05 or your prize will be transferred to someone else...

In [39]:
import $ivy.`com.github.tototoshi::scala-csv:2.0.0`

import scala.util.Using
import com.github.tototoshi.csv.CSVWriter

val textHeader = "Text"
val labelHeader = "Label"

def writeToCsv(path: String, dataset: Dataset): Unit = {
  val headers = Vector(textHeader, labelHeader)

  Using.resource(CSVWriter.open(path)) { writer =>
    val rows = dataset.map {
      case SmsSpamRecord(text, isSpam) => Vector(text, if (isSpam) "1" else "0")
    }
    writer.writeAll(headers +: rows)
  }
}

val trainingCsv = "data/training.csv"
writeToCsv(trainingCsv, training)
val validationCsv = "data/validation.csv"
writeToCsv(validationCsv, validation)
val testCsv = "data/test.csv"
writeToCsv(testCsv, test)

import $ivy.$
import scala.util.Using
import com.github.tototoshi.csv.CSVWriter
textHeader: String = "Text"
labelHeader: String = "Label"
defined function writeToCsv
trainingCsv: String = "data/training.csv"
validationCsv: String = "data/validation.csv"
testCsv: String = "data/test.csv"

In [24]:
Magic.!("pip", "install", "tiktoken==0.7.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [27]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import me.shadaj.scalapy.py
import py.SeqConverters

val tiktoken = py.module("tiktoken")

val tokenizer = tiktoken.get_encoding("gpt2")
val endOfTextToken = "<|endoftext|>"
val encodedEndOfTextToken = tokenizer.encode(endOfTextToken, allowed_special = py.Dynamic.global.set(Seq(endOfTextToken).toPythonProxy))
println(encodedEndOfTextToken)

[50256]


import $ivy.$
import me.shadaj.scalapy.py
import py.SeqConverters
tiktoken: py.Module = <module 'tiktoken' from '/usr/local/lib/python3.12/site-packages/tiktoken/__init__.py'>
tokenizer: py.Dynamic = <Encoding 'gpt2'>
endOfTextToken: String = "<|endoftext|>"
encodedEndOfTextToken: py.Dynamic = [50256]

In [47]:
Magic.!("pip", "install", "torch==2.4.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [71]:
import com.github.tototoshi.csv.CSVReader
import py.PyQuote

val torch = py.module("torch")

type Tokenizer = py.Dynamic

// Workaround to define a class that inherits from a Python class
py.exec {
  s"""from torch.utils.data import Dataset
     |
     |class SpamDataset(Dataset):
     |  def __init__(self, init):
     |    init(self)
     |
     |  def __getitem__(self, index):
     |    return self.getItem(index)
     |  
     |  def __len__(self):
     |    return self.len()
     |""".stripMargin
}
def SpamDataset(
  csvPath: String,
  tokenizer: Tokenizer,
  maxLength: Option[Int] = None,
  paddingTokenId: Int = 50_256
): py.Dynamic = {
  val smsSpamRecords = Using.resource(CSVReader.open(csvPath)) { csvReader =>
    csvReader.iteratorWithHeaders.map { row =>
      SmsSpamRecord(text = row(textHeader), isSpam = row(labelHeader).toInt > 0)
    }.toVector
  }
  val encodedTexts = {
    val encodedTexts = smsSpamRecords.map(_.text).map(tokenizer.encode(_).as[Seq[Int]].toVector)
    val padToLength = maxLength.getOrElse(encodedTexts.map(_.length).max)
    encodedTexts.map(_.padTo(padToLength, paddingTokenId))
  }
    
  val init = (self: py.Dynamic) => {
    self.maxLength = encodedTexts.head.length
    
    val getItem = (index: Int) => {
      val textTensor = torch.tensor(encodedTexts(index).toPythonProxy, dtype = torch.long)
      val labelTensor = torch.tensor(if (smsSpamRecords(index).isSpam) 1 else 0, dtype = torch.long)
      (textTensor, labelTensor)
    }
    self.getItem = getItem

    val len = () => smsSpamRecords.size
    self.len = len
  }
  py.Dynamic.global.SpamDataset(init)
}

import com.github.tototoshi.csv.CSVReader
import py.PyQuote
torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>
defined type Tokenizer
defined function SpamDataset

In [72]:
val trainingDataset = SpamDataset(trainingCsv, tokenizer)
val validationDataset = SpamDataset(validationCsv, tokenizer, maxLength = Some(trainingDataset.maxLength.as[Int]))
val testDataset = SpamDataset(testCsv, tokenizer, maxLength = Some(trainingDataset.maxLength.as[Int]))

trainingDataset: py.Dynamic = <SpamDataset object at 0xffff537eec90>
validationDataset: py.Dynamic = <SpamDataset object at 0xffff44718e00>
testDataset: py.Dynamic = <SpamDataset object at 0xffff447186b0>

In [73]:
val batchSize = 8
torch.manual_seed(123)

val trainingDataLoader = torch.utils.data.DataLoader(
  dataset = trainingDataset, 
  batch_size = batchSize,
  shuffle = true,
  num_workers = 0,
  drop_last = true
)
val validationDataLoader = torch.utils.data.DataLoader(
  dataset = validationDataset, 
  batch_size = batchSize,
  num_workers = 0,
  drop_last = false
)
val testDataLoader = torch.utils.data.DataLoader(
  dataset = testDataset, 
  batch_size = batchSize,
  num_workers = 0,
  drop_last = false
)

println(s"${py.Dynamic.global.len(trainingDataLoader)} training batches")
println(s"${py.Dynamic.global.len(validationDataLoader)} validation batches")
println(s"${py.Dynamic.global.len(testDataLoader)} test batches")

130 training batches
19 validation batches
38 test batches


batchSize: Int = 8
res73_1: py.Dynamic = <torch._C.Generator object at 0xffff7932c7f0>
trainingDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff44718860>
validationDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff52d7c8c0>
testDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff52d7cc50>